<a href="https://colab.research.google.com/github/hadwin-357/Protein_Language_Model/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Basic template from http://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F

In [3]:
#simple example, initial word with random embedding
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["world"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.8574, -1.4671,  0.2248, -0.5640, -0.2610]],
       grad_fn=<EmbeddingBackward0>)


In [5]:
#For N-gram language model problem, we need to calculate P(wi |wi-1, wi-2,...,wi-n+1) meaning based previous N words to predict next one
CONTEXT_SIZE =4
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print(ngrams[:3])


[(['shall', 'winters', 'forty', 'When'], 'besiege'), (['besiege', 'shall', 'winters', 'forty'], 'thy'), (['thy', 'besiege', 'shall', 'winters'], 'brow,')]


In [9]:
#turn words into tokens
vocab = set(test_sentence)  #unique words
word_to_ix = {word: i for i, word in enumerate(vocab)}
print(len(vocab))

97


In [7]:
#Ngram model
#vocab_size

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [8]:
losses = []  #store loss
loss_function = nn.NLLLoss()  # use negative log likelihood for loss for C classes classification
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)
model

NGramLanguageModeler(
  (embeddings): Embedding(97, 10)
  (linear1): Linear(in_features=40, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)

In [11]:
#train 10 epochs to get updated embeddding
for epoch in range(100):
    total_loss = 0
    for context, target in ngrams:
      #word to id
      context_idxs=torch.tensor([word_to_ix[word] for word in context], dtype=torch.long)

      #
      model.zero_grad()
      # run the forward pass
      log_probs=model(context_idxs)
      #calculate loss
      loss=loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
      #backward pass
      loss.backward()
      optimizer.step()

      #accumlate loss
      total_loss+=loss.item()

    losses.append(total_loss)
print([losses[i:i+10] for i in range(0, len(losses), 10)])

[[514.0127351284027, 511.0838289260864, 508.17943716049194, 505.2962031364441, 502.43221521377563, 499.5862486362457, 496.7554974555969, 493.93602657318115, 491.12773728370667, 488.32742381095886], [485.5379807949066, 482.75587272644043, 479.9818994998932, 477.2129735946655, 474.4465823173523, 471.68224596977234, 468.9203290939331, 466.1556191444397, 463.38957810401917, 460.61987566947937], [457.8502745628357, 455.0769956111908, 452.30245065689087, 449.5209069252014, 446.7336826324463, 443.9388825893402, 441.13389444351196, 438.323605298996, 435.50444293022156, 432.6759443283081], [429.8401737213135, 426.9927637577057, 424.13128328323364, 421.25888752937317, 418.3757655620575, 415.479421377182, 412.5707688331604, 409.6470694541931, 406.70846366882324, 403.75625002384186], [400.78994059562683, 397.8087764978409, 394.8140366077423, 391.799050450325, 388.76966524124146, 385.7201861143112, 382.65227019786835, 379.566859126091, 376.46604239940643, 373.35042810440063], [370.2136745452881, 36

In [12]:
# show updated embedding
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([ 1.8517,  1.4112, -1.0225,  1.2978,  0.4566,  0.7078, -0.2010,  0.6162,
         0.7890, -1.1715], grad_fn=<SelectBackward0>)


In [13]:
#excise
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


In [20]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix).view((1,-1)).shape

torch.Size([1, 4])

In [53]:
losses=[]
class CBOW(nn.Module):

    def __init__(self, voc_size, embedding_dim):
      super(CBOW, self).__init__()
      self.embedding=nn.Embedding(vocab_size, embedding_dim)
      self.linear1=nn.Linear(embedding_dim, vocab_size)


    def forward(self, inputs):
      lookup_embed =self.embedding(inputs)  #[4, 10 ]
      sum_embed=lookup_embed.sum(dim=0) #[1, 10]
      out =self.linear1(sum_embed)  #[1, 49]
      log_probs = F.log_softmax(out.view((1,-1)), dim=1)
      return log_probs


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)



model_1 = CBOW(vocab_size, EMBEDDING_DIM)
loss_function = nn.NLLLoss()  # use negative log likelihood for loss for C classes classification
optimizer = optim.SGD(model_1.parameters(), lr=0.001)
for epoch in range(10):
    total_loss = 0
    for context, target in data:
      #word to id
      context_idxs=make_context_vector(context, word_to_ix)  #

      model_1.zero_grad()
      # run the forward pass
      log_probs=model_1(context_idxs)
      #calculate loss
      loss=loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
      #backward pass
      loss.backward()
      optimizer.step()

      #accumlate loss
      total_loss+=loss.item()
    losses.append(total_loss)

In [54]:
print(losses)

[256.8255398273468, 254.13644981384277, 251.492959856987, 248.8937953710556, 246.3377504348755, 243.8236985206604, 241.3505585193634, 238.91731667518616, 236.52300465106964, 234.16668152809143]


In [56]:
model_1

CBOW(
  (embedding): Embedding(49, 10)
  (linear1): Linear(in_features=10, out_features=49, bias=True)
)

In [57]:
model_1.embedding.weight[word_to_ix["computer"]]

tensor([-0.8262,  0.9923,  0.0312,  0.2740, -0.7022,  0.6615, -0.6166,  0.2628,
         1.6284, -0.1836], grad_fn=<SelectBackward0>)